In [1]:
#import dependencies
import numpy as np
import pandas as pd
import requests
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from scipy.stats import linregress
import scipy.stats as st
import os

In [2]:
# Read single year crime files and store into final data frame
nba14_csv = "Resources/2014_nba.csv"
nba15_csv = "Resources/2015_nba.csv"
nba16_csv = "Resources/2016_nba.csv"

# pd.read_csv('file.csv', index_col=0)
nba14_df = pd.read_csv(nba14_csv, index_col=0)
nba15_df = pd.read_csv(nba15_csv, index_col=0)
nba16_df = pd.read_csv(nba16_csv, index_col=0)


In [3]:
#Combine all three csv files
frames = [nba14_df, nba15_df, nba16_df]
combine_df = pd.concat(frames)
combine_df

,Player,Year,Draft pick,Height (No Shoes),Height (With Shoes),Wingspan,Standing reach,Vertical (Max),Vertical (Max Reach),Vertical (No Step),Vertical (No Step Reach),Weight,Body Fat,Hand (Length),Hand (Width),Bench,Agility,Sprint
0,Aaron Craft,2014,NaN,72.75,73.75,74.00,94.5,32.0,126.5,29.0,123.5,192.0,6.1,7.75,8.50,NaN,10.78,3.26
1,Aaron Gordon,2014,4.0,79.50,80.75,83.75,105.0,39.0,144.0,32.5,137.5,220.0,5.1,8.75,10.50,NaN,10.81,3.27
2,Adreian Payne,2014,15.0,81.00,81.75,88.00,109.0,NaN,NaN,NaN,NaN,239.0,7.6,9.25,9.50,NaN,NaN,NaN
3,Alec Brown,2014,50.0,84.25,85.25,85.50,109.0,30.0,139.0,27.0,136.0,231.0,7.7,9.25,8.75,NaN,12.05,3.48
4,Alex Kirk,2014,NaN,81.25,82.75,87.50,109.5,31.5,141.0,28.5,138.0,252.0,13.6,9.25,10.50,NaN,11.20,3.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Troy Williams,2016,NaN,77.75,78.75,80.25,102.0,36.0,138.0,30.0,132.0,218.0,10.0,8.25,9.25,NaN,10.72,3.35
57,Tyler Ulis,2016,34.0,68.75,70.00,74.00,88.5,38.0,126.5,32.5,121.0,149.0,5.2,7.50,7.50,NaN,10.80,3.20
58,Wade Baldwin,2016,17.0,74.50,76.00,83.25,100.0,38.0,138.0,32.5,132.5,202.0,5.2,9.00,9.75,NaN,10.45,3.19
59,Wayne Selden,2016,NaN,76.25,77.75,82.50,101.0,NaN,NaN,NaN,NaN,232.0,7.3,9.00,9.25,NaN,NaN,NaN


In [4]:
#Drop the draft pick NA values
combine_df = combine_df[combine_df['Draft pick'].notna()]
combine_df

,Player,Year,Draft pick,Height (No Shoes),Height (With Shoes),Wingspan,Standing reach,Vertical (Max),Vertical (Max Reach),Vertical (No Step),Vertical (No Step Reach),Weight,Body Fat,Hand (Length),Hand (Width),Bench,Agility,Sprint
1,Aaron Gordon,2014,4.0,79.50,80.75,83.75,105.0,39.0,144.0,32.5,137.5,220.0,5.1,8.75,10.50,NaN,10.81,3.27
2,Adreian Payne,2014,15.0,81.00,81.75,88.00,109.0,NaN,NaN,NaN,NaN,239.0,7.6,9.25,9.50,NaN,NaN,NaN
3,Alec Brown,2014,50.0,84.25,85.25,85.50,109.0,30.0,139.0,27.0,136.0,231.0,7.7,9.25,8.75,NaN,12.05,3.48
6,C.J. Wilcox,2014,28.0,75.50,77.00,81.75,100.5,37.5,138.0,31.0,131.5,201.0,6.5,8.75,9.00,NaN,11.30,3.22
7,Cameron Bairstow,2014,49.0,80.75,81.75,84.75,107.0,33.5,140.5,31.0,138.0,252.0,8.7,8.75,9.00,NaN,11.39,3.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,Taurean Prince,2016,12.0,78.50,79.75,83.50,102.0,36.0,138.0,34.0,136.0,220.0,6.5,9.00,8.50,NaN,10.96,3.24
55,Thon Maker,2016,10.0,83.75,84.75,87.00,110.5,36.5,147.0,32.0,142.5,216.0,6.1,8.75,9.50,NaN,11.15,3.33
57,Tyler Ulis,2016,34.0,68.75,70.00,74.00,88.5,38.0,126.5,32.5,121.0,149.0,5.2,7.50,7.50,NaN,10.80,3.20
58,Wade Baldwin,2016,17.0,74.50,76.00,83.25,100.0,38.0,138.0,32.5,132.5,202.0,5.2,9.00,9.75,NaN,10.45,3.19


In [5]:
#Connect to local database
rds_connection_string = "postgres:postgres@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [6]:
#Rename all columns so they populate in SQL database
combine_df.rename(columns={'Player': 'player', 'Year': 'year', 'Draft pick': 'draft_pick', \
                           'Height (No Shoes)': 'height_no_shoes', 'Height (With Shoes)': 'height_with_shoes',\
                           'Wingspan': 'wingspan', 'Standing reach': 'standing_reach', 'Vertical (Max)': 'vertical_max', \
                           'Vertical (Max Reach)': 'vertical_max_reach', 'Vertical (No Step)': 'vertical_no_step', \
                           'Vertical (No Step Reach)': 'vertical_no_step_reach', 'Weight': 'weight', 'Body Fat':'body_fat',\
                           'Hand (Length)': 'hand_length', 'Hand (Width)': 'hand_width', 'Bench':'bench', 'Agility': 'agility', 'Sprint': 'sprint'}, inplace=True)

C:\Users\Janelle Goddard\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
#Print new dataframe w/ updated title names
combine_df

,player,year,draft_pick,height_no_shoes,height_with_shoes,wingspan,standing_reach,vertical_max,vertical_max_reach,vertical_no_step,vertical_no_step_reach,weight,body_fat,hand_length,hand_width,bench,agility,sprint
1,Aaron Gordon,2014,4.0,79.50,80.75,83.75,105.0,39.0,144.0,32.5,137.5,220.0,5.1,8.75,10.50,NaN,10.81,3.27
2,Adreian Payne,2014,15.0,81.00,81.75,88.00,109.0,NaN,NaN,NaN,NaN,239.0,7.6,9.25,9.50,NaN,NaN,NaN
3,Alec Brown,2014,50.0,84.25,85.25,85.50,109.0,30.0,139.0,27.0,136.0,231.0,7.7,9.25,8.75,NaN,12.05,3.48
6,C.J. Wilcox,2014,28.0,75.50,77.00,81.75,100.5,37.5,138.0,31.0,131.5,201.0,6.5,8.75,9.00,NaN,11.30,3.22
7,Cameron Bairstow,2014,49.0,80.75,81.75,84.75,107.0,33.5,140.5,31.0,138.0,252.0,8.7,8.75,9.00,NaN,11.39,3.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,Taurean Prince,2016,12.0,78.50,79.75,83.50,102.0,36.0,138.0,34.0,136.0,220.0,6.5,9.00,8.50,NaN,10.96,3.24
55,Thon Maker,2016,10.0,83.75,84.75,87.00,110.5,36.5,147.0,32.0,142.5,216.0,6.1,8.75,9.50,NaN,11.15,3.33
57,Tyler Ulis,2016,34.0,68.75,70.00,74.00,88.5,38.0,126.5,32.5,121.0,149.0,5.2,7.50,7.50,NaN,10.80,3.20
58,Wade Baldwin,2016,17.0,74.50,76.00,83.25,100.0,38.0,138.0,32.5,132.5,202.0,5.2,9.00,9.75,NaN,10.45,3.19


In [8]:
#Create another clean dataframe dropping the Bench column
new_combine_df =combine_df.drop(columns=['bench'])
new_combine_df

,player,year,draft_pick,height_no_shoes,height_with_shoes,wingspan,standing_reach,vertical_max,vertical_max_reach,vertical_no_step,vertical_no_step_reach,weight,body_fat,hand_length,hand_width,agility,sprint
1,Aaron Gordon,2014,4.0,79.50,80.75,83.75,105.0,39.0,144.0,32.5,137.5,220.0,5.1,8.75,10.50,10.81,3.27
2,Adreian Payne,2014,15.0,81.00,81.75,88.00,109.0,NaN,NaN,NaN,NaN,239.0,7.6,9.25,9.50,NaN,NaN
3,Alec Brown,2014,50.0,84.25,85.25,85.50,109.0,30.0,139.0,27.0,136.0,231.0,7.7,9.25,8.75,12.05,3.48
6,C.J. Wilcox,2014,28.0,75.50,77.00,81.75,100.5,37.5,138.0,31.0,131.5,201.0,6.5,8.75,9.00,11.30,3.22
7,Cameron Bairstow,2014,49.0,80.75,81.75,84.75,107.0,33.5,140.5,31.0,138.0,252.0,8.7,8.75,9.00,11.39,3.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,Taurean Prince,2016,12.0,78.50,79.75,83.50,102.0,36.0,138.0,34.0,136.0,220.0,6.5,9.00,8.50,10.96,3.24
55,Thon Maker,2016,10.0,83.75,84.75,87.00,110.5,36.5,147.0,32.0,142.5,216.0,6.1,8.75,9.50,11.15,3.33
57,Tyler Ulis,2016,34.0,68.75,70.00,74.00,88.5,38.0,126.5,32.5,121.0,149.0,5.2,7.50,7.50,10.80,3.20
58,Wade Baldwin,2016,17.0,74.50,76.00,83.25,100.0,38.0,138.0,32.5,132.5,202.0,5.2,9.00,9.75,10.45,3.19


In [9]:
#Use pandas to load csv converted DataFrame into database
new_combine_df.to_sql(name= 'new_combine_df', con=engine, if_exists='append', index=False)

In [10]:
#Print table name created in SQL
engine.table_names() 

['new_combine_df']

In [11]:
#Confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from new_combine_df', con=engine).head()

,player,year,draft_pick,height_no_shoes,height_with_shoes,wingspan,standing_reach,vertical_max,vertical_max_reach,vertical_no_step,vertical_no_step_reach,weight,body_fat,hand_length,hand_width,agility,sprint
0,Aaron Gordon,2014,4,79.50,80.75,83.75,105.0,39.0,144.0,32.5,137.5,220.0,5.1,8.75,10.50,10.81,3.27
1,Adreian Payne,2014,15,81.00,81.75,88.00,109.0,NaN,NaN,NaN,NaN,239.0,7.6,9.25,9.50,NaN,NaN
2,Alec Brown,2014,50,84.25,85.25,85.50,109.0,30.0,139.0,27.0,136.0,231.0,7.7,9.25,8.75,12.05,3.48
3,C.J. Wilcox,2014,28,75.50,77.00,81.75,100.5,37.5,138.0,31.0,131.5,201.0,6.5,8.75,9.00,11.30,3.22
4,Cameron Bairstow,2014,49,80.75,81.75,84.75,107.0,33.5,140.5,31.0,138.0,252.0,8.7,8.75,9.00,11.39,3.37
